# fGSEA benchmark

This jupyter notebook calculates the speed of fGSEA as well as the p-value consistency. We install the newest version of fGSEA from github.

In [64]:
library(devtools)
install_github("ctlab/fgsea")

Loading required package: usethis




glue (1.5.1    -> 1.6.0   ) [CRAN]
BH   (1.75.0-0 -> 1.78.0-0) [CRAN]


Installing 2 packages: glue, BH

Installing packages into ‘/home/maayanlab/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



✔  checking for file ‘/tmp/RtmpnZeecz/remotes1365b06aa9d141/ctlab-fgsea-37924a6/DESCRIPTION’
─  preparing ‘fgsea’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘fgsea_1.21.0.tar.gz’
   


Installing package into ‘/home/maayanlab/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



In [1]:
library(fgsea)
library(data.table)
library(ggplot2)

library(BiocParallel)
register(SerialParam())

In [47]:
sig = read.table("test/aging_muscle_gtex.tsv", sep=",", header=T)
signature = sig[,2]
names(signature) = sig[,1]

## Initializing gene set library

The gene set library can be downloaded from Enrichr.

In [45]:
ll = readLines("test/GO_Biological_Process_2021.txt")
genes = list()
gmt = list()
for(l in ll){
    sp = unlist(strsplit(l, "\t"))
    pname = sp[1]
    gmt[[pname]] = sp[3:length(sp)]
}

## Run fGSEA

In [48]:
st = Sys.time()
fgseaRes <- fgsea(pathways = gmt,
                  stats    = signature,
                  eps      = 0.0,
                  minSize  = 5,
                  maxSize  = 4000)

print(Sys.time() - st)

Time difference of 12.75591 secs


# Consistency

Compare consistency of log p-values.

In [16]:
ll = readLines("test/KEGG_2019_Mouse.txt")
genes = list()
gmt = list()
for(l in ll){
    sp = unlist(strsplit(l, "\t"))
    pname = sp[1]
    gmt[[pname]] = sp[3:length(sp)]
}

In [ ]:
res = list()
for(i in 1:10){
    fgseaRes <- fgsea(pathways = gmt,
                      stats    = signature,
                      minSize  = 5,
                      maxSize  = 4000
                      )
    res[[length(res)+1]] = log(fgseaRes[,"pval"])
}

## Calulate error

The calculated p-value error will be used in the p-value consistency benchmark notebook to compare GSEApy, blitzGSEA, and fGSEA

In [44]:
pp=do.call(cbind, res)
print(dim(pp))
ee=(1-cor(pp))
diag(ee) = NaN
paste(unlist(rowMeans(ee,  na.rm=T)), collapse=",")


[1] 296  10


[1] "0.00261250354614742,0.00274601095318889,0.00268698398862981,0.00254896856733903,0.0028239254670881,0.00255107983368626,0.00266219682016753,0.00248607221489015,0.00294916957896707,0.00300309821889459"